In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re
import pickle
from tqdm import tqdm
from constants import base_path
import matplotlib.pyplot as plt
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
tqdm.pandas()

In [2]:
with open(f"{base_path}\\input\\release_conditions.json") as f:
  disease_dict = json.load(f)
disease_list = list(disease_dict.keys())

In [3]:
with open(f"{base_path}\\input\\release_evidences.json") as f:
  evidences = json.load(f)
evidences_list = []
evidences_dict = {}
evidences_en_to_code = {}
for e in evidences.keys():
  # only binary symptoms and no antecedents
  if (not evidences[e]["possible-values"]) and (not evidences[e]["is_antecedent"]):
    evidences_list.append(e)
    evidences_dict[e] = evidences[e]["question_en"]
    evidences_en_to_code[evidences[e]["question_en"]] = e
evidences_code_to_en = evidences_dict
evidences_list_en = list(evidences_en_to_code.keys())
evidences_dict["AGE"] = "AGE"
evidences_dict["SEX"] = "SEX"
feature_columns = ["AGE", "SEX"] + evidences_list

In [4]:
def get_next_question(evidences, questionnaire, feature_embeddings_df):
    centroid = np.array([feature_embeddings_df.loc[e].values for e in evidences]).mean(axis=0)
    _, indices = questionnaire.kneighbors([centroid])
    ask_list = [evidences_list_en[i] for i in indices[0] if evidences_list_en[i] not in evidences]
    try:
        return ask_list
    except:
        return []

In [5]:
def get_evidences(answers, user_evidences, questionnaire, feature_embeddings_df, n_questions):
    ask = True
    question_counter = 1 # counts initial evidence sa q0
    evidences_en = [evidences_code_to_en[e] for e in user_evidences]
    answers_en = [evidences_code_to_en[e] for e in answers if e in evidences_list]
    asked = evidences_en.copy()
    while question_counter < n_questions:
      ask =  True
      next_question_idx = 0
      next_question = get_next_question(evidences_en, questionnaire, feature_embeddings_df)
      while ask and (question_counter < n_questions):
        if next_question[next_question_idx] not in asked:
          answer = 1 if next_question[next_question_idx] in answers_en else 0
          asked.append(next_question[next_question_idx])
          question_counter+=1
          if answer==1:
            evidences_en.append(next_question[next_question_idx])
            ask = False
          else:
            next_question_idx += 1
          if next_question_idx > n_questions:
            break
        else:
            next_question_idx += 1
    return [evidences_en_to_code[e] for e in evidences_en]

In [6]:
def data_proc(df, questionnaire, feature_embeddings_df, n_questions):
    df["binary_evidences_all"] = df["EVIDENCES"].apply(lambda x: [d for d in ast.literal_eval(x) if d in evidences_list])
    df["binary_evidences_all_count"] = df["binary_evidences_all"].apply(lambda x: len(x))
    df["binary_evidences"] = df[["EVIDENCES", "INITIAL_EVIDENCE"]].progress_apply(lambda x: get_evidences(ast.literal_eval(x[0]), [x[1]], questionnaire, feature_embeddings_df, n_questions), axis=1)
    df["binary_evidences_count"] = df["binary_evidences"].apply(lambda x: len(x))
    df["hit_rate"] = df["binary_evidences_count"]/df["binary_evidences_all_count"]
    hit_rate = df["hit_rate"]
    for e in evidences_list:
        df[e] = df["binary_evidences"].apply(lambda x: 1 if e in x else 0)
    df["SEX"] = df["SEX"].map({'F': 0, 'M': 1})
    ftr_df = df[feature_columns + ["PATHOLOGY"]]
    questionnaire_df = df[["binary_evidences_all", "binary_evidences", "INITIAL_EVIDENCE"]]
    return ftr_df, hit_rate, questionnaire_df

In [7]:
def pred(x):
    pred_list = []
    for i in range(len(disease_list)):
        if x[i] > 0:
            pred_list.append({
                "disease": disease_list[i],
                "probability": x[i]})
    if pred_list:
        pred_df = pd.DataFrame(pred_list).set_index('disease')
        # return only top 1 - allows ties
        pred_df['rank'] = pred_df['probability'].rank(method='min', ascending=False)
        pred_df = pred_df.sort_values(by="rank")
        pred_df = pred_df[pred_df["rank"]<=1][["probability"]]
        return list(pred_df.index)
    else:
        return []

In [8]:
def get_missed_evidence(actual, asked):
    return list(set(actual)^set(asked))

In [9]:
def validate(x):
    return [x[0]]==x[1]

In [10]:
with open(f'{base_path}\\output\\questionnaire\\questionnaire.pkl', 'rb') as f:
    questionnaire = pickle.load(f)
feature_embeddings_df = pd.read_pickle(f'{base_path}\\output\\questionnaire\\questionnaire_embeddings.pkl')

In [11]:
diagnosis_df_valid = pd.read_csv(f"{base_path}\\input\\release_validate_patients")

In [12]:
data_proc(diagnosis_df_valid.head(5), questionnaire, feature_embeddings_df, n_questions=20)

100%|██████████| 5/5 [00:00<00:00, 168.53it/s]


(   AGE  SEX  E_91  E_53  E_159  E_129  E_154  E_155  E_210  E_140  ...  E_193  \
 0   55    0     0     1      0      0      1      0      0      1  ...      0   
 1   10    0     0     1      0      0      0      1      0      0  ...      0   
 2   68    0     1     1      0      1      0      0      0      0  ...      0   
 3   13    1     0     1      0      0      0      0      0      0  ...      0   
 4   48    1     0     1      0      0      0      0      0      0  ...      0   
 
    E_168  E_180  E_67  E_171  E_111  E_182  E_103  E_23     PATHOLOGY  
 0      0      0     0      0      0      0      0     0        Anemia  
 1      0      0     0      1      1      0      0     0  Panic attack  
 2      0      0     0      0      0      0      0     0     Influenza  
 3      0      0     0      0      0      0      0     0        Anemia  
 4      0      0     0      0      0      0      0     0     Boerhaave  
 
 [5 rows x 99 columns],
 0    1.000000
 1    1.000000
 2    1.0000

In [13]:
questionnaire_dict = {}
n_questions_list = list(range(1, 31))
for n_questions in n_questions_list:
    diagnosis_df_valid = pd.read_csv(f"{base_path}\\input\\release_validate_patients")
    print(f"Asking {n_questions} questions...")
    diagnosis_df_valid, hit_rate, questionnaire_df = data_proc(diagnosis_df_valid, questionnaire, feature_embeddings_df, n_questions)
    questionnaire_dict[f"questionnaire_data_{n_questions}"] = {
        "diagnosis_df_valid": diagnosis_df_valid,
        "hit_rate": hit_rate,
        "questionnaire_df": questionnaire_df
    }

Asking 1 questions...


100%|██████████| 132448/132448 [00:07<00:00, 17119.25it/s]


Asking 2 questions...


100%|██████████| 132448/132448 [02:16<00:00, 970.27it/s] 


Asking 3 questions...


...

100%|██████████| 132448/132448 [11:26<00:00, 193.00it/s]


Asking 29 questions...


100%|██████████| 132448/132448 [09:52<00:00, 223.38it/s]


Asking 30 questions...


100%|██████████| 132448/132448 [20:02<00:00, 110.17it/s]


In [14]:
metrics_dict = {"Random_forest": {}, "Logistic_regression":{}, "Decision_Tree":{}}

## Random Forest

In [15]:
model_dict = {}
for disease in disease_list:
    disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
    with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{disease_filename}_model.pkl', 'rb') as f:
        model_dict[disease] = pickle.load(f)

In [16]:
for n_questions in n_questions_list:
    diagnosis_df_valid = questionnaire_dict[f"questionnaire_data_{n_questions}"]["diagnosis_df_valid"]
    hit_rate = questionnaire_dict[f"questionnaire_data_{n_questions}"]["hit_rate"]
    questionnaire_df = questionnaire_dict[f"questionnaire_data_{n_questions}"]["questionnaire_df"]
    print(f"using {n_questions} questions predicting diagnosis...")
    for disease in disease_list:
        rf_model = model_dict[disease]
        diagnosis_df_valid[disease] = rf_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
    diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)
    diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)
    diagnosis_df_valid["hit_rate"] = hit_rate
    print("validating...")
    acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
    acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
    acc["mean_hit_rate"] = np.mean(hit_rate)
    hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
    acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)
    metrics_dict["Random_forest"][f"n_questions_{n_questions}"] = acc
        


predicting diagnosis...


100%|██████████| 132448/132448 [00:02<00:00, 61331.66it/s]


validating...
predicting diagnosis...


100%|██████████| 132448/132448 [00:03<00:00, 43442.78it/s]


validating...
predicting diagnosis...


...

validating...
predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 115908.30it/s]


validating...
predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 106680.66it/s]

validating...


In [33]:
plt_y_rf = [metrics_dict["Random_forest"][f"n_questions_{i}"]["error_rate"] for i in n_questions_list]
plt.plot(n_questions_list, plt_y_rf, marker='o')
plt.xlabel('number of questions')
plt.ylabel('misdiagnosis rate')
plt.title('Random Forest - Questionnaire Experiments')
plt.savefig(f'{base_path}\\output\\error_analysis_questionnaire\\random_forest_experiments.jpg', bbox_inches='tight')
plt.clf()

## Logistic Regression

In [21]:
model_dict = {}
for disease in disease_list:
    disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
    with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{disease_filename}_logreg_model.pkl', 'rb') as f:
        model_dict[disease] = pickle.load(f)

In [22]:
for n_questions in n_questions_list:
    diagnosis_df_valid = questionnaire_dict[f"questionnaire_data_{n_questions}"]["diagnosis_df_valid"]
    hit_rate = questionnaire_dict[f"questionnaire_data_{n_questions}"]["hit_rate"]
    questionnaire_df = questionnaire_dict[f"questionnaire_data_{n_questions}"]["questionnaire_df"]
    print(f"using {n_questions} questions predicting diagnosis...")
    for disease in disease_list:
        rf_model = model_dict[disease]
        diagnosis_df_valid[disease] = rf_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
    diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)
    diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)
    diagnosis_df_valid["hit_rate"] = hit_rate
    print("validating...")
    acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
    acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
    acc["mean_hit_rate"] = np.mean(hit_rate)
    hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
    acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)
    metrics_dict["Logistic_regression"][f"n_questions_{n_questions}"] = acc

using 1 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 68009.36it/s]


validating...
using 2 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 78091.07it/s]


validating...
using 3 questions predicting diagnosis...


...

validating...
using 29 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 120851.14it/s]


validating...
using 30 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:03<00:00, 37029.58it/s]


validating...


In [32]:
plt_y_logreg = [metrics_dict["Logistic_regression"][f"n_questions_{i}"]["error_rate"] for i in n_questions_list]
plt.plot(n_questions_list, plt_y_logreg, marker='o')
plt.xlabel('number of questions')
plt.ylabel('misdiagnosis rate')
plt.title('Logistic Regression - Questionnaire Experiments')
plt.savefig(f'{base_path}\\output\\error_analysis_questionnaire\\logistic_regression_experiments.jpg', bbox_inches='tight')
plt.clf()

## Decision Tree

In [27]:
model_dict = {}
for disease in disease_list:
    disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
    with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{disease_filename}_dt_model.pkl', 'rb') as f:
        model_dict[disease] = pickle.load(f)

In [28]:
for n_questions in n_questions_list:
    diagnosis_df_valid = questionnaire_dict[f"questionnaire_data_{n_questions}"]["diagnosis_df_valid"]
    hit_rate = questionnaire_dict[f"questionnaire_data_{n_questions}"]["hit_rate"]
    questionnaire_df = questionnaire_dict[f"questionnaire_data_{n_questions}"]["questionnaire_df"]
    print(f"using {n_questions} questions predicting diagnosis...")
    for disease in disease_list:
        rf_model = model_dict[disease]
        diagnosis_df_valid[disease] = rf_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
    diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)
    diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)
    diagnosis_df_valid["hit_rate"] = hit_rate
    print("validating...")
    acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
    acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
    acc["mean_hit_rate"] = np.mean(hit_rate)
    hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
    acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)
    metrics_dict["Decision_Tree"][f"n_questions_{n_questions}"] = acc

using 1 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:03<00:00, 42265.38it/s]


validating...
using 2 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 90102.49it/s]


validating...
using 3 questions predicting diagnosis...


...

validating...
using 29 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 120338.58it/s]


validating...
using 30 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 92367.01it/s] 

validating...


In [30]:
plt_y_dt = [metrics_dict["Decision_Tree"][f"n_questions_{i}"]["error_rate"] for i in n_questions_list]
plt.plot(n_questions_list, plt_y_dt, marker='o')
plt.xlabel('number of questions')
plt.ylabel('misdiagnosis rate')
plt.title('Decision Tree - Questionnaire Experiments')
plt.savefig(f'{base_path}\\output\\error_analysis_questionnaire\\decision_tree_experiments.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [37]:
plt.plot(n_questions_list, plt_y_rf, marker='o', label = "Random Forest")
plt.plot(n_questions_list, plt_y_logreg, marker='o', label = "Logistic Regression")
plt.plot(n_questions_list, plt_y_dt, marker='o', label = "Decision Tree")
plt.xlabel('number of questions')
plt.ylabel('misdiagnosis rate')
plt.title('Questionnaire Experiments')
plt.ylim(0, 1)
plt.legend()
plt.savefig(f'{base_path}\\output\\error_analysis_questionnaire\\questionnaire_experiments.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [35]:
with open(f"{base_path}\\output\\error_analysis_questionnaire\\validation_metric_questionnaire_experiments.json", "w") as outfile: 
    json.dump(metrics_dict, outfile, indent=True)

In [ ]:
# ran for approx 920 min
# 49 x 3 x 30 = 4410 model runs hahahaha